In [ ]:
PSEUDO CODE:

# go to page (get numeric_id and name_before, contained in link)
go to http://mountainproject.com/v/[climb_name]/[climb_id_numeric]

# get keys
parse link by '/' 
last entry in array is numeric id
second to last is climb name

# separate desciptions for this climb from left sidebar
get all inside 'div' with id 'rspCol800'

# get guidebook rating (this needs to be adjusted for bouldering and for ice, maybe use bold style?)
find first 'span' tag with class 'rateYDS'
go inside 'a' tag and find ';' in text, pull string after up to '"'
if pulled text is of regex format 5.[1-14][a-d]:
    store it as guidebook rating

# get average star rating
find 'span' with 'id' equal to 'routeStars'
get 'script' tag inside that 'span'
get text and find 'starsHtml(', get decimal between that and next ','
if  1 < decimal-1 < 4:
    store as average rating
else:
    say average rating has no solution

### get table with main descriptive data
get to 'table' below 'span' with id 'starSummaryText', next or sibling method

## get climb types, pitches, feet, and grade
find 'td' with text 'Type*' and get next 'td' text
parse on commas, store as array

# climb types
for type in [trad, ice, alpine, sport, boulder]:
    if find in array:
        make type cell = 1

# pitches
if find regex"* pitches" in array:
    strip that element of non-numeric chars     
    if number in range(0,100):
        pitches = number
    else:
        pitches = NaN
else:
    pitches = NaN

# feet
if find element with regex "*'":
    strip non-numeric characters
    if number in range (0,15000):
        feet = number

# grade 
if find element with regex 'Grade*':
    parse entry on space
    take last element of parse
    if element in [I, II, III, IV, V]:
        grade = element
    else:
        grade = NaN

## get consensus rating (this will have to be adapted for boulder and ice)
find 'td' with text 'Consensus*' and get next 'td'
within that 'td', get 'span' with class 'rateYDS'
if text of that span matches regex 5.[1-14][a-d]:
    store as consensus rating:
else:
    consensus rating = unknown

## get consensus rating (this will have to be adapted for boulder and ice)
find 'td' with text 'FA*' and get next 'td' text
if find regex '19[0-9][0-9]':
    append match as fa_year

## get consensus rating (this will have to be adapted for boulder and ice)
find 'td' with text 'Page Views*' and get next 'td' text
convert to number
page_views = number


#### get user climb data
go to http://mountainproject.com/scripts/ShowObjectStats.php?id=[climb_key_numeric]

### get star quality votes (this data either needs to be updated or added)
find 'span' with text 'Star Quality Votes'
go to next sibling 'small' tag
go to next sibling 'table' tag
go to child 'tbody' tad and extract all children

def starqualityfunc(td1, td2):
    get 'a' tag from td1 children
    user = 'href' attribute
    climb = current_climb
    get 'script' tag inside 'td' tag and find regex for "starsHtml([2-5]," 
    star_rating = number from regex 
    append trio to user_climb table
    
    
for 'tr' in 'tbody' children:
    if contains 4 elements:
        spit into two sets of two elements
        apply star quality votes func to each subset
    if contains 2 elements:
        apply star quality func to them

        
### get suggested ratings
find 'span' with text 'Suggested Ratings'
go to next sibling 'small' tag
go to next sibling 'table' tag
go to child 'tbody' tag and extract all children
for 'tr' in children:
        
def suggratfunc(td1, td2):
    get 'a' tag from td1 children
    user = 'href' attribute (parsed)
    climb = current_climb
    get 'span' tag inside 'td' tag with class rateYDS 
    star_rating = that span's text'
    append trio to user_climb table
    
    
for 'tr' in 'tbody' children:
    if contains 4 elements:
        spit into two sets of two elements
        apply star quality votes func to each subset
    if contains 2 elements:
        apply star quality func to them
        
### get ticks
find 'span' with text 'Ticks'
go to next sibling 'small' tag
go to next sibling 'table' tag
go to child 'tbody' tag and extract all children
for 'tr' in children:
    get href of 'a' tag of first 'td' element
    store it as user
    store climb as climb
    go to nextg 'td' tag, store text as date
    append date to user_climb entry in user_climb_table

    
